<a href="https://colab.research.google.com/github/xuhu357/DataAnalysis/blob/master/ch07_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%80%EB%B9%84_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

### Index Merge 

머지하려는 키가 DataFrame의 색인(index)일 수 있다. 

이때에는 left_index=True 혹은 right_index=True 옵션을 지정해 해당 색인을 머지 키로 사용할 수 있다.

In [0]:
left1 = DataFrame({
    'key': ['a', 'b', 'a', 'a', 'b', 'c'],
    'value': range(6)
})

right1 = DataFrame({
    'group_val': [3.5, 7]
}, index=['a', 'b'])

In [3]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [4]:
right1

,group_val
a,3.5
b,7.0


In [5]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


위에서 보다 싶이, merge는 기본적으로 inner join 즉 교집합을 구하지만, outer join을 통해서 합집합 처리 할 수 있다.

In [6]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


**계층적 색인**은 조금 복잡하다. 그러나, 천천히 하나하나씩 살펴보면 어렵지는 않다~

In [0]:
lefth = DataFrame({
    'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002], 
    'data': np.arange(5)
})

righth = DataFrame(
    np.arange(12).reshape((6, 2)),
    index = [
        ['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ],
    columns = ['event1', 'event2']
)

In [15]:
lefth

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


In [16]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

이럴 때에는 리스트로 여러 칼럼을 지정해서 머지해야 한다. 

중복되는 색인 값을 다룰때에는 주의해야 함. 

In [17]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11
3,3,Nevada,2001,0,1


발견 했겠지만, left_on의 길이와 right_index의 길이가 같아야 merge가 가능하다. 



---

outer merge방식으로 한번 해보자~

In [18]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4.0,5.0
0,0.0,Ohio,2000,6.0,7.0
1,1.0,Ohio,2001,8.0,9.0
2,2.0,Ohio,2002,10.0,11.0
3,3.0,Nevada,2001,0.0,1.0
4,4.0,Nevada,2002,NaN,NaN
4,NaN,Nevada,2000,2.0,3.0


지금까지 본것은 left에는 column의 내용, 그리고 right에는 index 내용을 merge하는 과정이다. 

---

만약 양쪽에 공통으로 있는 여러개의 index를 merge한다면, 어떻게 할까? 좀 더 쉽다~

In [0]:
left2 = DataFrame(
    [[1., 2.], [3., 4.], [5., 6.]],
    index=['a', 'c', 'e'],
    columns=['Ohio', 'Nevada']
)

In [0]:
right2 = DataFrame(
    [
        [7., 8.], [9., 10.], [11., 12.], [13., 14.]
    ],
    index=['b', 'c', 'd', 'e'],
    columns=['Missouri', 'Alabama']
)

In [22]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [23]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


두 dataframe의 색인에 관해서 merge를 해보자. 방식은 outer

In [24]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


색인으로 머지 할 때, DataFrame의 join 메소드를 사용하면, 더 간편하다. 

join 메소드는 칼럼이 겹치지 않으며, 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용할 수 있다.


---

위의 예제를 고쳐쓰면, 

In [25]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [26]:
left2.join(right2) # outer로 지정해주지 않으면, default는 inner join이다.

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


DataFrame의 join 메소드는 ***왼쪽 조인***을 수행.

join메소드를 호출한 DataFrame의 칼럼중 하나에 대해서 조인을 수행하는것도 가능하다. 

left1의 key에 대해서 right1과 join을 해보자.

In [27]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [28]:
right1

,group_val
a,3.5
b,7.0


In [29]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


색인 대 색으로 두 DataFrame을 합치려면, 그냥 간단히 merge하려는 DataFrame의 리스트를 join 메소드에 넘기면 된다.

하지만, 일반적으로 이런 머지는 concat메소드를 사용한다~

In [0]:
another = DataFrame(
    [[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon']
)

left2에 right2 와 another DataFrame을 join해보자~

In [31]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [32]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0
